## Wpierw należy zaimportować bibliotekę `tensorflow` i `numpy`.

Przygotowuję zbiory danych

In [1]:
import numpy as np # fort init arry
from numpy import log as ln # for ln
from pandas import DataFrame # for dataframes
import math # sin

In [2]:
x_set = np.array(list(range(1, 30)), dtype=float)
y_set = np.array([0.25, 0.5, 0.75, 1], dtype=float)

In [3]:
print(x_set)
print(y_set)

[ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17. 18.
 19. 20. 21. 22. 23. 24. 25. 26. 27. 28. 29.]
[0.25 0.5  0.75 1.  ]


In [4]:
def function_a(x, y):
    return y * ln(x)

def function_b(x, y):
    return 2 * y * math.sin((x-1)/20)

In [5]:
function_a(5, 0.5)

0.8047189562170501

In [6]:
function_b(5,0.5)

0.19866933079506122

In [7]:
list_a = []

for x in list(x_set):
  for y in list(y_set):
    data = {}
    data['x'] = x
    data['y'] = y
    data['result'] = function_a(x,y)
    list_a.append(data)
    
df_a = DataFrame(list_a)
    
list_b = []

for x in list(x_set):
  for y in list(y_set):
    data = {}
    data['x'] = x
    data['y'] = y
    data['result'] = function_b(x,y)
    list_b.append(data)

df_b = DataFrame(list_b)

In [8]:
df_a.head(5)

,x,y,result
0,1.0,0.25,0.000000
1,1.0,0.50,0.000000
2,1.0,0.75,0.000000
3,1.0,1.00,0.000000
4,2.0,0.25,0.173287


In [9]:
df_b.head(5)

,x,y,result
0,1.0,0.25,0.00000
1,1.0,0.50,0.00000
2,1.0,0.75,0.00000
3,1.0,1.00,0.00000
4,2.0,0.25,0.02499


In [10]:
#from sklearn.cross_validation import train_test_split
from sklearn.model_selection import train_test_split

feature_col_names = ['x', 'y']
predicted_class_names = ['result']
split_test_size = 0.30

In [11]:
X_a = df_a[feature_col_names].values     # predictor feature columns (8 X m)
y_a = df_a[predicted_class_names].values # predicted class (1=true, 0=false) column (1 X m)

X_a_train, X_a_test, y_a_train, y_a_test = train_test_split(X_a, y_a, test_size=split_test_size, random_state=42) 
                            # test_size = 0.3 is 30%, 42 is the answer to everything

print("{0:0.2f}% in training set of a".format((len(X_a_train)/len(df_a.index)) * 100))
print("{0:0.2f}% in test set of a".format((len(X_a_test)/len(df_a.index)) * 100))

69.83% in training set of a
30.17% in test set of a


In [12]:
X_b = df_b[feature_col_names].values     # predictor feature columns (8 X m)
y_b = df_b[predicted_class_names].values # predicted class (1=true, 0=false) column (1 X m)

X_b_train, X_b_test, y_b_train, y_b_test = train_test_split(X_b, y_b, test_size=split_test_size, random_state=42) 
                            # test_size = 0.3 is 30%, 42 is the answer to everything

print("{0:0.2f}% in training set of b".format((len(X_b_train)/len(df_b.index)) * 100))
print("{0:0.2f}% in test set of b".format((len(X_b_test)/len(df_b.index)) * 100))

69.83% in training set of b
30.17% in test set of b
